# How to build a network?
- Social network simulation?

- We have:
    - Datetime
    - Innovation
    - Location
- How to build a Network from this?
- -> Tree shaped?

# CSV Import
- Dask only import if one of the Inno Columns is ==True; 
- Loop trough all CSV's and append "tures" to a single dd
- Turn into gpd

In [2]:
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
import os
# Treat '' and np.inf as NaN
pd.options.mode.use_inf_as_na = True

ERROR 1: PROJ: proj_create_from_database: Open of /home/linusrg/Code/LILI/.micromamba/envs/LILI-Env/share/proj failed


In [3]:
path = '~/Code/LILI/data/' 
cols = ["inno_01","inno_02","inno_03","inno_04","inno_05","inno_06","inno_07","inno_08","inno_09","inno_10","inno_12","inno_13","inno_15","inno_16","inno_17","inno_18","inno_19","inno_20","inno_21","inno_22","inno_23","inno_24","inno_25","inno_26","inno_27","inno_29","inno_30","inno_31","inno_32","inno_33","inno_34","inno_39"]
imp_cols = ["created_at", "twitter_id","user_id","user_location","longitude","latitude","place_box_1_long","place_box_1_lat","place_box_2_long","place_box_2_lat","country","country_code","place_full_name","place_id","place_name"] + cols

In [4]:
dtypes={'twitter_id': 'int64',
        'created_at': 'object',
        'user_id': 'int64',
        'user_location': 'string',
        'longitude': 'float',
        'latitude': 'float',
        'place_box_1_long': 'string',
        'place_box_1_lat': 'string',
        'place_box_2_long': 'string',
        'place_box_2_lat': 'string',
        'country': 'string',
        'country_code': 'string',
        'place_full_name': 'string',
        'place_id': 'string',
        'place_name': 'string',
        'place_type': 'string',
        'inno_01': 'boolean',
        'inno_02': 'boolean',
        'inno_03': 'boolean',
        'inno_04': 'boolean',
        'inno_05': 'boolean',
        'inno_06': 'boolean',
        'inno_07': 'boolean',
        'inno_08': 'boolean',
        'inno_09': 'boolean',
        'inno_10': 'boolean',
        'inno_12': 'boolean',
        'inno_13': 'boolean',
        'inno_15': 'boolean',
        'inno_16': 'boolean',
        'inno_17': 'boolean',
        'inno_18': 'boolean',
        'inno_19': 'boolean',
        'inno_20': 'boolean',
        'inno_21': 'boolean',
        'inno_22': 'boolean',
        'inno_23': 'boolean',
        'inno_24': 'boolean',
        'inno_25': 'boolean',
        'inno_26': 'boolean',
        'inno_27': 'boolean',
        'inno_29': 'boolean',
        'inno_30': 'boolean',
        'inno_31': 'boolean',
        'inno_32': 'boolean',
        'inno_33': 'boolean',
        'inno_34': 'boolean',
        'inno_39': 'boolean'}

In [55]:
def DaskImp(path, imp_cols=imp_cols):
        df = dd.read_csv(path, 
                sep = '\t', 
                lineterminator='\n', 
                usecols=imp_cols, 
                false_values =['', ' ', "''", "' '", "", " ", 'False', 'Fal', "['', '', '']" ], #There is some weired stuff in that Data! Turning it into false Values. 
                dtype= dtypes
                )
        return df        

In [7]:
# Stack all the bool to one named column
def BoolStack(df_s):
    df_s['inno_nr'] = df_s[cols].idxmax(1).astype('string')
    df_n = df_s.drop(cols, axis=1)
    df_n['inno_nr'] = df_n['inno_nr'].astype('string')
    return(df_n)

In [59]:
#create files in subfolders list
subfolders = []
for root, dirs, files in os.walk('../data'):
   for name in files:
      if root in subfolders:
         pass
      else:
         subfolders.append(root) 
subfolders.sort()


In [8]:
#Create empty csv with actual headers to initialize dask data frame
# The empty.csv is included in the repo (unlike the data folder)
#df_s = df_i.head(0)
#df_s.to_csv('empty.csv', sep = "\t")

In [63]:
#loop trough folders, parallel import in folders (parallelization across subfolders not possible with Dask, takes ~5s)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#ignore the warning, append still works and concat is not implemented in dask yet :)

#Create empty header df to append to
df_s = BoolStack(DaskImp('empty.csv'))

#Loop trough folders
for folder in subfolders:
     # Import a CSV, stack the Boolean Innovation Columns to one Column
     df = BoolStack(DaskImp(f'{folder}/*.csv'))
     # Append to the previous, but only columns with an innovation 
     df_s = df_s.append(df[df.inno_nr != 0])


In [64]:
print('Number of Columns: ', len(df_s.columns))

Number of Columns:  16
Number of Columns:  16


In [68]:
df_s.shape

(Delayed('int-cb205802-6d8a-4949-b1e8-6ffc5e64c4c9'), 16)

In [ ]:
#turn dask data frame into pandas data frame
#and export this as parquet
df = df_s.compute()
df.to_parquet('geocoding_all_parallel_0425.parquet')

In [ ]:
len(df)

# Get coordinates for Tweets (Geocoding)
- Define one spatial identifier for every user (Coordinate, Place, User Location)
- Safe user and place in a separate list
- 
- Geocode each unique location

- Slight noise for tweets at same city?



# Visualize
--> Make a time based visualization for each Innovation

- Kepler.GL?